# Template for erosion rate calculations based on 10Be concentrations

Last updated by Simon M. Mudd on 03/02/2023

This is a template you can use to calculate erosion rates from 10Be concentrations in detrital stream sediments. It uses CAIRN (https://doi.org/10.5194/esurf-4-655-2016) to make these computations. 

You will need to enter your own data but otherwise just follow the template and you will soon be computing erosion rates. 

## If you are on colab

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdviztools`. 

Note that if we install `condacolab` first the `lsdviztools` installation fails. 

In [ ]:
!pip install lsdviztools

Now we need to install lsdtopotools. We do this using something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now use mamba to install `lsdtopotools`. This will install the command line tools. This step takes a bit over a minute. 

In [ ]:
!mamba install lsdtopotools &> /dev/null

In [ ]:
!lsdtt-basic-metrics

## Get the climate data required for the analysis

We need to get some data that is used to compute pressure for our samples. This affects the 10Be production rate. 

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP2.bin", "NCEP2.bin")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP_hgt.bin", "NCEP_hgt.bin")

## Getting topographic data of your study area from OpenTopography.org

OpenTopograhy (https://www.opentopography.org/) is a great place to download topographic data. 

It has vast quantities of lidar data that you can download from the site. In addition there is an API for downloading global datasets (at a range of resolutions). 

`lsdviztools` has a script that can download this global data and convert it to the file format and coordinate reference system (CRS) you need for analysis in `lsdtopotools`. 

**To do this you need to have a user account at OpenTopography.** You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

Once you have done that, you should get your API key and paste it into a text file somewhere safe (if any digital security experts want to explain how to better do this I am all ears). You will need to point to that file when calling the `lsdviztools` opentopography scraper. 

If you are working in *colab* then you should upload the file into your working directory. 

* You need to add the lower left and upper right hand coordinates, in lat-long, of your study area. 
* You can do this by right clicking and copying coordinates on google maps
* Also set a data prefix name
* And decide on the data type: options here https://portal.opentopography.org/apidocs/#/Public/getGlobalDem, we use COP30

In [4]:
lower_left_coords = [23.020977925808698, -110.07075635034748], 
upper_right_coords = [23.403400882011553, -109.7686323359646]
Dataset_name = "baja"
source_name = "COP30"

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

study_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left_coords, 
                        upper_right_coordinates = upper_right_coords,
                        prefix = Dataset_name, 
                        api_key_file = your_OT_api_key_file)
study_DEM.print_parameters()
study_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_name+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## Make sure the data is in the correct place

Right, lets see what this place looks like:

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%matplotlib inline
Base_file = r_prefix
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

## Now for some parameter files for the analysis

We need a few data files to run the analysis. You probably don't need to change any of these unless you are an advanced user. 

In [7]:
CRN_param_fname = "./"+Dataset_name+"_"+source_name +"_UTM.CRNParam"
CRN_rasters_fname = "./"+Dataset_name+"_"+source_name +"_UTM_CRNRasters.csv"

with open(CRN_param_fname, 'w', encoding="utf-8") as f:
    f.write('min_slope: 0.0001\n')
    f.write('source_threshold: 12\n')
    f.write('search_radius_nodes: 4\n')
    f.write('threshold_stream_order: 2\n')
    f.write('theta_step: 30\n')
    f.write('phi_step: 30\n')
    f.write('Muon_scaling: BraucherBorchers\n')

with open(CRN_rasters_fname, 'w', encoding="utf-8") as f:
    f.write("./"+Dataset_name+"_"+source_name +"_UTM")
    f.write('\n')